In [1]:
import pprint
from types import SimpleNamespace
from OptiHPLCHandler import EmpowerHandler, EmpowerInstrumentMethod
from typing import List, Optional, Union, Dict

In [2]:
# get api address from .env file
import os
from dotenv import load_dotenv
load_dotenv('../.env_vars')
EMPOWER_API_ADDRESS = os.getenv('EMPOWER_API_ADDRESS')

In [3]:
# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)

handler.connection.default_get_timeout=300
handler.connection.default_post_timeout=300

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = "20240416_test"
    full_method = handler.GetInstrumentMethod(method_name)

In [4]:
# make sample set table for robustness testing
from generate_basic_robustness_methods import generate_basic_robustness_instrument_methods
# generate methods and return the dict of methods
dict_methods = generate_basic_robustness_instrument_methods(handler, full_method)
dict_methods

{'input_method': {'method_name': '20240416_test', 'run_time': '20.00'},
 'rampup': {'method_name': '20240416_test_ramp', 'run_time': '10'},
 'scaled': {'method_name': '20240416_test_cond_10m', 'run_time': '10.0'},
 'plus_strong': {'method_name': '20240416_test_1_0pct', 'run_time': '20.00'},
 'minus_strong': {'method_name': '20240416_test_m1_0pct', 'run_time': '20.00'},
 'iso_start_1': {'method_name': '20240416_test_iso_15m_0', 'run_time': '35.0'},
 'iso_start_2': {'method_name': '20240416_test_iso_30m_0', 'run_time': '50.0'},
 'plus_temp': {'method_name': '20240416_test_2_5C', 'run_time': '20.00'},
 'minus_temp': {'method_name': '20240416_test_m2_5C', 'run_time': '20.00'},
 'rampdown': {'method_name': '20240416_test_low', 'run_time': '1'}}

In [6]:
from generate_basic_robustness_methods import generate_basic_robustness_sampleset, sample_set_namer, post_and_run_experiment

plate = "ANSI-48Vial2mLHolder"
plates = {"1": plate, "2": plate}
system = "12_5208_HCLASS_BIN"
node = "Epdkhqr01048"
project = "WebAPI_test"
method_name = "20240416_test"

sample_list = generate_basic_robustness_sampleset(dict_methods)
sample_set_method_name = sample_set_namer(method_name)

# post and run experiment
post_and_run_experiment(
        handler=handler,
        sample_set_method_name=sample_set_method_name,
        sample_list=sample_list,
        plates=plates,
        system=system,
        node=node,
        post=True,
        run=False
    )


Logging in to Empower...
Log in successful.
Posting SampleSet Method 20240416_test_robustness...
Sample Set Method 20240416_test_robustness posted.
Sample Set Method 20240416_test_robustness not run.


In [39]:
dict_methods={
    "input_method": {
        "method_name": "test",
        "run_time": "20.0",            
                     },
    "rampup": {
        "method_name": "test_rampup",
        "run_time": "10.0",            
                     },
    "plus_strong": {
        "method_name": "test_plus_strong",
        "run_time": "20.0",            
                     },
    "minus_strong": {
        "method_name": "test_minus_strong",
        "run_time": "20.0",            
                     },
    "iso_start_1": {
        "method_name": "test_iso_start_1",
        "run_time": "50.0",            
                     },
    "iso_start_2": {
        "method_name": "test_iso_start_2",
        "run_time": "80.0",            
                     },
    "plus_temp": {
        "method_name": "test_plus_temp",
        "run_time": "20.0",            
                     },
    "minus_temp": {
        "method_name": "test_minus_temp",
        "run_time": "20.0",            
                     },
}

In [9]:
# add wash and re-equilibration
from types import SimpleNamespace
from empower_implementation.empower_tools import determine_strong_eluent
method=SimpleNamespace(
gradient_table = [
    {"Time": 0, "CompositionA": 95, "CompositionB": 5, "CompositionC": 0, "CompositionD": 0},
    {"Time": 10, "CompositionA": 50, "CompositionB": 50, "CompositionC": 0, "CompositionD": 0},
],
method_name = "test",
)

wash_strong_composition = 95
wash_time = 2
re_equilibration_time = 7

strong_eluent, list_weak_eluents = determine_strong_eluent(method.gradient_table)

gradient_portion_end_time = method.gradient_table[-1]["Time"]
gradient_start_composition_strong = method.gradient_table[0][strong_eluent]



wash_rows = [{
    "Time": gradient_portion_end_time + 0.1,
    strong_eluent: wash_strong_composition,
},
{
    "Time": gradient_portion_end_time + wash_time + 0.1,
    strong_eluent: wash_strong_composition,
},
{
    "Time": gradient_portion_end_time + wash_time + 0.2,
    strong_eluent: gradient_start_composition_strong,
},
{
    "Time": gradient_portion_end_time + wash_time + re_equilibration_time,
    strong_eluent: gradient_start_composition_strong,
}]

method.gradient_table += wash_rows

method.gradient_table






[{'Time': 0,
  'CompositionA': 95,
  'CompositionB': 5,
  'CompositionC': 0,
  'CompositionD': 0},
 {'Time': 10,
  'CompositionA': 50,
  'CompositionB': 50,
  'CompositionC': 0,
  'CompositionD': 0},
 {'Time': 10.1, 'CompositionB': 95},
 {'Time': 12.1, 'CompositionB': 95},
 {'Time': 12.2, 'CompositionB': 5},
 {'Time': 19, 'CompositionB': 5}]